# Copyright 2024 Samarth Singh samarths@bu.edu

"""
# Blackjack Helper System: Training and Real-Time Card Recognition

## Purpose:
This script builds a Blackjack helper system that:
1. Trains a ResNet18 model to recognize playing cards (Ace, 2-10, Jack, Queen, King).
2. Provides real-time card recognition using OpenCV.
3. Dynamically calculates card values and suggests the next move in Blackjack ("Hit", "Stay", or "Blackjack").

## Workflow:
1. **Dataset Download**:
   - The script automatically downloads the `cards-image-datasetclassification` dataset from KaggleHub.
   - The dataset is split into training, validation, and testing sets.

2. **Training**:
   - A pretrained ResNet18 model (from ImageNet) is fine-tuned on the card dataset.
   - The model predicts card classes (`13` classes for cards: Ace, 2-10, Jack, Queen, King).
   - Best-performing model is saved as `resnet_best_model.pth`.

3. **Testing**:
   - Evaluates the model on the test dataset.
   - Outputs the overall accuracy of the model.

4. **Real-Time Card Recognition**:
   - Captures live video using OpenCV.
   - Draws an ROI box where the player places the card.
   - Recognizes the card inside the ROI when the spacebar is pressed.
   - Updates the player's total card value.
   - Suggests the next move in Blackjack: "Hit", "Stay", or "Blackjack".
   - If "Stay" is suggested, it shows the total value and stops processing further cards.

---

## Libraries Used:
1. **PyTorch**:
   - `torch`: Core library for deep learning models.
   - `torchvision`: Provides pre-trained ResNet18 and transformations for image data.

2. **OpenCV**:
   - Used for capturing video, defining the Region of Interest (ROI), and real-time display of predictions.

3. **TQDM**:
   - Provides progress bars for training and testing loops.

4. **PIL**:
   - Converts OpenCV images into a format suitable for PyTorch preprocessing.

5. **Matplotlib**:
   - Displays pop-up suggestions for Blackjack moves.

6. **KaggleHub**:
   - Downloads the card dataset from Kaggle with a single command.

---

## How to Use:
1. Install required libraries:
   ```bash
   pip install torch torchvision tqdm opencv-python matplotlib kagglehub


In [ ]:
# Required libraries
import os
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.models import resnet18
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import kagglehub  # For downloading the dataset

In [ ]:
# ----------------------------------------------
# Step 1: Download the Dataset
# ----------------------------------------------

print("Downloading dataset...")
# Downloads the dataset from KaggleHub and sets the dataset path
dataset_path = kagglehub.dataset_download("gpiosenka/cards-image-datasetclassification")
print("Path to dataset files:", dataset_path)

# Define dataset folder paths for train, validation, and test sets
train_dir = os.path.join(dataset_path, "train")
val_dir = os.path.join(dataset_path, "valid")
test_dir = os.path.join(dataset_path, "test")

In [ ]:
# ----------------------------------------------
# Step 2: Define Data Transformations
# ----------------------------------------------

# Data transformations for training: Augmentation techniques are applied
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resizes images for ResNet
    transforms.RandomHorizontalFlip(),  # Adds random horizontal flips
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),  # Adjust brightness, contrast
    transforms.RandomRotation(15),  # Random rotation for diversity
    transforms.ToTensor(),  # Converts image to tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalizes to match ImageNet stats
])

# Data transformations for validation and test: No augmentation, just resizing and normalization
test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# ----------------------------------------------
# Step 3: Load Dataset and DataLoader
# ----------------------------------------------

# Create datasets for training, validation, and testing
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(val_dir, transform=test_transform)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)

# Create DataLoader for each dataset
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Get the class names from the dataset
class_names = train_dataset.classes
print(f"Classes: {class_names}")
num_classes = len(class_names)  # Number of output classes (13 for cards)

In [ ]:
# ----------------------------------------------
# Step 4: Load ResNet18 Model
# ----------------------------------------------

# Load ResNet18 pretrained on ImageNet
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet18(weights="IMAGENET1K_V1")

# Modify the final fully connected layer to match the number of card classes
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)  # Send the model to the available device (CPU/GPU)

# Set up optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with learning rate 0.001
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification

In [ ]:
# ----------------------------------------------
# Step 5: Train the Model
# ----------------------------------------------

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    """
    Function to train the ResNet model on the training dataset.
    Args:
        model: ResNet18 model.
        train_loader: DataLoader for the training dataset.
        val_loader: DataLoader for the validation dataset.
        criterion: Loss function.
        optimizer: Optimizer.
        num_epochs: Number of training epochs.
    """
    best_accuracy = 0.0  # Tracks the best validation accuracy
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0  # Track cumulative loss
        correct_train = 0  # Track correct predictions
        total_train = 0  # Track total samples

        # Loop through the training dataset
        train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", unit="batch")
        for images, labels in train_loop:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()  # Reset gradients
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update model parameters
            running_loss += loss.item()

            # Calculate accuracy
            _, preds = torch.max(outputs, 1)
            correct_train += (preds == labels).sum().item()
            total_train += labels.size(0)
            train_loop.set_postfix(loss=loss.item())

        train_accuracy = correct_train / total_train

        # Validate the model
        model.eval()  # Set model to evaluation mode
        correct_val = 0
        total_val = 0
        val_loop = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]", unit="batch")
        with torch.no_grad():
            for images, labels in val_loop:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                correct_val += (preds == labels).sum().item()
                total_val += labels.size(0)
                val_loop.set_postfix(accuracy=correct_val / total_val)

        val_accuracy = correct_val / total_val
        print(f"Epoch {epoch+1}/{num_epochs}, Train Accuracy: {train_accuracy:.4f}, Val Accuracy: {val_accuracy:.4f}")

        # Save the model if validation accuracy improves
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            model_save_path = "resnet_best_model.pth"
            torch.save(model.state_dict(), model_save_path)
            print(f"Model saved at: {model_save_path}")

    print("Training complete. Best accuracy: {:.4f}".format(best_accuracy))

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)


In [ ]:
# ----------------------------------------------
# Step 6: Test the Model
# ----------------------------------------------

def test_model(model, test_loader):
    """
    Function to test the trained model on the test dataset.
    Args:
        model: Trained ResNet18 model.
        test_loader: DataLoader for the test dataset.
    """
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.4f}")

# Load the best model and test it
model.load_state_dict(torch.load("resnet_best_model.pth"))
test_model(model, test_loader)

In [ ]:
# ----------------------------------------------
# Step 7: Real-Time Card Recognition with OpenCV
# ----------------------------------------------

# Real-time card recognition logic
# Set up optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_accuracy = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", unit="batch")
        for images, labels in train_loop:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            correct_train += (preds == labels).sum().item()
            total_train += labels.size(0)
            train_loop.set_postfix(loss=loss.item())

        train_accuracy = correct_train / total_train

        # Validation
        model.eval()
        correct_val = 0
        total_val = 0
        val_loop = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]", unit="batch")
        with torch.no_grad():
            for images, labels in val_loop:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                correct_val += (preds == labels).sum().item()
                total_val += labels.size(0)
                val_loop.set_postfix(accuracy=correct_val / total_val)

        val_accuracy = correct_val / total_val
        print(f"Epoch {epoch+1}/{num_epochs}, Train Accuracy: {train_accuracy:.4f}, Val Accuracy: {val_accuracy:.4f}")

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            torch.save(model.state_dict(), "resnet_best_model.pth")
    print("Training complete. Best accuracy: {:.4f}".format(best_accuracy))

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

# Test function
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.4f}")

# Load the best model and test it
model.load_state_dict(torch.load("resnet_best_model.pth"))
print("Trained model saved at: resnet_best_model.pth")
test_model(model, test_loader)


In [ ]:
# Real-time card recognition using OpenCV
model.eval()  # Set model to evaluation mode

def get_card_value(card_name, current_total):
    if card_name == "Ace":
        return 11 if current_total + 11 <= 21 else 1
    return int(card_name)

def blackjack_suggestion(total):
    if total == 21:
        return "Blackjack! Stay!"
    elif total >= 17:
        return "Stay"
    else:
        return "Hit"

# OpenCV part
cap = cv2.VideoCapture(0)
player_total = 0
last_card_label = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    box_start, box_end = (200, 100), (450, 350)
    cv2.rectangle(frame, box_start, box_end, (0, 255, 0), 2)

    if last_card_label:
        cv2.putText(frame, f"Last Card: {last_card_label}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    if cv2.waitKey(1) & 0xFF == ord(' '):
        roi = frame[box_start[1]:box_end[1], box_start[0]:box_end[0]]
        image = Image.fromarray(roi).convert("RGB")
        image = test_transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            outputs = model(image)
            _, pred = torch.max(outputs, 1)
            last_card_label = class_names[pred.item()]
            player_total += get_card_value(last_card_label, player_total)

        suggestion = blackjack_suggestion(player_total)
        print(f"Suggestion: {suggestion}")

cap.release()
cv2.destroyAllWindows()
